# Evaluating the Retriever & End-to-End System
> A review of Information Retrieval and the role it plays in a QA system

- title: "Evaluating the Retriever & End-to-End System"
- toc: true 
- badges: true
- comments: true
- hide: true
- permalink: /hidden/
- search_exclude: false
- categories:

In our last post, [Evaluating QA: Metrics, Predictions, and the Null Response](https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html), we took a deep dive look at how to asses the quality of a BERT-like Reader for Question Answering (QA) using the Hugging Face framework. In this post, we'll focus on the former component of an end-to-end QA system - the Retriever. Specifically, we'll introduce Elasticsearch as a powerful and efficient Information Retrieval (IR) tool that can be used to scour through large corpora and retrieve relevant documents. Through the post, we'll explain how to implement and evaluate a Retriever in the context of Question Answering and demonstrate the impact it has on an end-to-end QA system.

### Prerequisites
* a basic understanding of Information Retrieval & Search
* a basic understanding of IR based QA systems (see previous posts)
* a basic understanding of Transformers and PyTorch
* a basic understanding of the SQuAD2.0 dataset

# Retrieving the right document is important


![](my_icons/michael_scott_quote.jpg "You miss 100% of the shots you don't take")


We believe that what Michael Scott really meant to say is:

> "***You miss 100% of the questions if the answer doesn't appear in the input context***"

As we have discussed throughout this blog series, many modern QA systems take a two-staged approach to answering questions. In the first stage, a document retriever selects $N$ potentially relevant documents from a given corpus. Subsequently, a machine comprehension model processes each of the $N$ documents to determine an answer to the input question. Because of recent advances in NLP and deep learning (i.e. flashy Transformer models), the machine comprehension component of question answering has typically been the main focus of evaluation. Stage one of these systems has recieved limited attention despite its obvious importance...stage two is bounded by performance at stage one. Let's get more specific.

We [recently explained methods]() that enable BERT-like models to produce robust answers given a question and context passage by selectively processing predictions and by refraining from answering certain questions at all. While the ability to properly comprehend a passage and produce a correct answer is a critical feature of any QA tool, the success of the overall system is highly dependent on first providing a correct passage to read through. Without being fed a context passage that actually contains the ground-truth answer, the overall system's performance is limited to how well it can predict no-answer questions. To demonstrate, we'll revisit an example from our [first blog post]() where three questions were asked of the Wikipedia search engine based QA system:

```
**Example 1: Incorrect**
Question: When was Barack Obama born?
Top wiki result: <WikipediaPage 'Barack Obama Sr.'>
Answer: 18 June 1936 / February 2 , 1961 / 

**Example 2: Correct**
Question: Why is the sky blue?
Top wiki result: <WikipediaPage 'Diffuse sky radiation'>
Answer: Rayleigh scattering / 

**Example 3: Correct**
Question: How many sides does a pentagon have?
Top wiki result: <WikipediaPage 'The Pentagon'>
Answer: five / 
```

In Example 1, the Reader had no chance of producing the correct answer because of its outright absence from the context served up by the Retriever. Namely, the Retriever erroneously provided a page about Barack Obama Sr. instead of his son, the former US President. In this case, the only way the Reader could have possibly produced the correct answer was if the correct answer was actually not to answer at all. On the flip side, in Example 3, the Retriever did not identify the globally "correct" document - it returned an article about "The Pentagon" instead of a page about geometry - but nonetheless, it provided enough context for the Reader to succeed.

These quick examples illustrate why an effective Retriever is crucial for an end-to-end QA system. Now let's take a deeper look at a classic tool used for information retrieval - Elasticsearch.

# Elasticsearch as an IR Tool

![](my_icons/elasticsearch-logo.png "Elasticsearch")

Modern QA systems employ a variety of techniques for the task of information retrieval ranging from traditional sparse vector word matching (ex. Elasticsearch) to [novel approaches](https://arxiv.org/pdf/2004.04906.pdf) using dense representations of encoded passages combined with [efficient search capabilities](https://github.com/facebookresearch/faiss). Despite the flurry of contemporary research efforts in this area, the traditional sparse vector approach performs very well overall and has only recently been overtaken by embedding-based systems for end-to-end QA retrieval tasks. For that reason, we'll explore Elasticsearch as an easy to use framework for document retrieval. So, what exactly is Elasticsearch?

Elasticsearch is a powerful open-source search and analytics engine built on the [Apache Lucene](https://lucene.apache.org/) library that is capable of handling all types of data including textual, numerical, geospatial, structrured, and unstructured. It is built to scale with a robust set of features, rich ecosystem, and diverse list of client libraries making it easy to integrate and use. In the context of information retrieval for automated question answering, we are keenly interested in the features surrounding full-text search. Elasticsearch provides a convenient way to index documents so they can quickly be queried for nearest neighbor search using a TF-IDF based similarity metric. Specifically, it uses [BM25](https://opensourceconnections.com/blog/2015/10/16/bm25-the-next-generation-of-lucene-relevation/) term weighting to represent question and context passages as high-dimensional, sparse vectors that are efficiently searched in an inverted index. Let's unpack those ideas a bit.


#### Inverted Index

The purpose of an inverted index is to store text in a data structure that allows for efficient and fast full-text searches. An inverted index is essentially just a mapping between unique terms and documents which contain those terms. For example, consider the following two documents and a depiction of an inverted index built from them:
1. "Elasticsearch is a powerful technique for search!"
2. "Manual search is a slow technique."

|      Term     | Document 1 | Document 2 |
|:-------------:|:----------:|:----------:|
|       a       |      x     |      x     |
| elasticsearch |      x     |            |
|      for      |      x     |            |
|       is      |      x     |      x     |
|     manual    |            |      x     |
|    powerful   |      x     |            |
|     search    |      x     |      x     |
|      slow     |            |      x     |
|   technique   |      x     |      x     |

Notice that the unique set of terms from both documents are contained in the index, and we can easily lookup which document contains which terms. Searching this inverted index for the phrase "search technique" would return both documents because both terms are present in each document, while searching for the phrase "powerful technique" would return only Document 1. Search itself is quite a bit more complicated than the boolean logic depicted here as it involves relevance scoring (among other query dependent logic), however this oversimplification is intended to demonstrate the quick and powerful nature of the inverted index data structure.

> Note: In the example above, all tokens have been lowercased and punctuation removed. This happens as part of an important preprocessing pipeline that we'll explain in more detail later in the post.

The inverted index representation is considered *sparse* by construction because for each document, you end up with a vector containing few terms relative to all terms in the index. This indexing process is what allows Elasticsearch to peruse large collections of text documents orders of magnitude faster than traditional SQL databases. While the exact match nature of search in this data structure is powerful and effective, it isn't without flaws. Word matching is limited in its ability to take semantically related concepts into search consideration. For example, consider the following question and context borrowed from the [DPR paper](https://arxiv.org/pdf/2004.04906.pdf):

> **Question:** "Who is the bad guy in lord of the rings?"\
> **Context:** "Sala Baker is an actor and stuntman from New Zealand. He is best known for portraying the villain Sauron in the Lord of the Rings trilogy..."

An exact match based system like Elasticsearch would struggle to retrieve this supporting context passage because it lacks the ability to relate the concepts of "bad guy" and "villain". Modern document retrieval systems that take advantage of learned, dense representations of text would perform better in this situation.

## Using Elasticsearch with SQuAD2.0

With this basic understanding of how Elasticsearch works, let's dive in and build our own Document Retrieval system by indexing a set of Wikipedia article paragraphs that support questions and answers from the SQuAD2.0 dataset. Before we get started, we'll need to download and prepare data from SQuAD2.0.

### Download and Prepare SQUAD2.0

In [64]:
# collapse-hide

# Download the SQuAD2.0 train & dev sets
!wget -P data/squad/ https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -P data/squad/ https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
    
import json

--2020-06-22 15:10:08--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘data/squad/dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  1.64MB/s    in 2.5s    

2020-06-22 15:10:11 (1.64 MB/s) - ‘data/squad/dev-v2.0.json’ saved [4370528/4370528]



The following `parse_qa_records` function will extract question/answer examples, as well as article content from the SQuAD2.0 data set. A common practice in IR for QA is to segment large articles into smaller passages before indexing for two main reasons:

1. **Transformer based Readers are very slow** - providing an entire Wikipedia article to BERT for processing takes a considerable amount of time and also defeats the purpose of using an IR tool to narrow the search space for the Reader.
2. **Smaller passages reduce noise** - by identifying a more concise context passage for BERT to read through, we reduce the chance of BERT getting lost.

Of course the chunking method proposed here doesn't come without a cost. Larger document size means each document contains more information. By reducing passage size, we are potentially trading off system recall for speed - though there are techniques to alleviate this as we will disucss later in the post.

With our chunking approach, each article paragraph will be prepended with the article title and collectively serve as a corpus of documents for which our Elasticsearch Retriever will search over. In practice, open-domain QA systems sit atop massive collections of documents (think all of Wikipedia) to provide a breadth of information to answer general-knowledge questions from. For the purposes of demonstrating Elasticsearch functionality, we will limit our corpus to only the Wikipedia articles supporting SQuAD2.0 questions.

In [14]:
def parse_qa_records(data):
    '''
    Loop through SQuAD2.0 dataset and parse out question/answer examples and unique article paragraphs
    
    Returns:
        qa_records (list) - question/answer examples as list of dictionaries
        wiki_articles (list) - unique Wikipedia titles and article paragraphs recreated from SQuAD data
    
    '''
    num_with_ans = 0
    num_without_ans = 0
    qa_records = []
    wiki_articles = {}
    
    for article in data:
        
        for i, paragraph in enumerate(article['paragraphs']):
            
            wiki_articles[article['title']+f'_{i}'] = article['title'] + ' ' + paragraph['context']
            
            for questions in paragraph['qas']:
                
                qa_record = {}
                qa_record['example_id'] = questions['id']
                qa_record['document_title'] = article['title']
                qa_record['question_text'] = questions['question']
                
                try: 
                    qa_record['short_answer'] = questions['answers'][0]['text']
                    num_with_ans += 1
                except:
                    qa_record['short_answer'] = ""
                    num_without_ans += 1
                    
                qa_records.append(qa_record)
        
        
    wiki_articles = [{'document_title':title, 'document_text': text}\
                         for title, text in wiki_articles.items()]
                
    print(f'Data contains {num_with_ans} question/answer pairs with a short answer, and {num_without_ans} without.'+
          f'\nThere are {len(wiki_articles)} unique wikipedia article paragraphs.')
                
    return qa_records, wiki_articles

In [89]:
# load and parse data
train_file = "data/squad/train-v2.0.json"
dev_file = "data/squad/dev-v2.0.json"

train = json.load(open(train_file, 'rb'))
dev = json.load(open(dev_file, 'rb'))

qa_records, wiki_articles = parse_qa_records(train['data'])
qa_records_dev, wiki_articles_dev = parse_qa_records(dev['data'])

Data contains 86821 question/answer pairs with a short answer, and 43498 without.
There are 19035 unique wikipedia article paragraphs.
Data contains 5928 question/answer pairs with a short answer, and 5945 without.
There are 1204 unique wikipedia article paragraphs.


In [90]:
# Show parsed record example
qa_records[10]

{'example_id': '56d43c5f2ccc5a1400d830ab',
 'document_title': 'Beyoncé',
 'question_text': 'What was the first album Beyoncé released as a solo artist?',
 'short_answer': 'Dangerously in Love'}

In [91]:
# Show example of parsed wiki article paragraph
print(wiki_articles[10])

{'document_title': 'Beyoncé_10', 'document_text': 'Beyoncé Beyoncé\'s first solo recording was a feature on Jay Z\'s "\'03 Bonnie & Clyde" that was released in October 2002, peaking at number four on the U.S. Billboard Hot 100 chart. Her first solo album Dangerously in Love was released on June 24, 2003, after Michelle Williams and Kelly Rowland had released their solo efforts. The album sold 317,000 copies in its first week, debuted atop the Billboard 200, and has since sold 11 million copies worldwide. The album\'s lead single, "Crazy in Love", featuring Jay Z, became Beyoncé\'s first number-one single as a solo artist in the US. The single "Baby Boy" also reached number one, and singles, "Me, Myself and I" and "Naughty Girl", both reached the top-five. The album earned Beyoncé a then record-tying five awards at the 46th Annual Grammy Awards; Best Contemporary R&B Album, Best Female R&B Vocal Performance for "Dangerously in Love 2", Best R&B Song and Best Rap/Sung Collaboration for "

### Download Elasticsearch

With our data ready to go, let's download, install, and configure Elasticsearch using one of the two following methods (Colab recommended). After executing the setup, we will have an Elasticsearch service running locally.


In [ ]:
# collapse-hide

# If using Colab - Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# collapse-hide

# If running locally - download, install, and launch Elasticsearch per https://www.elastic.co/downloads/elasticsearch

### Getting Data into Elasticsearch

We'll use the [official low-level Python client library](https://elasticsearch-py.readthedocs.io/en/master/) for interacting with Elasticsearch.

In [ ]:
# collapse-hide

!pip install elasticsearch
!pip install tqdm

By default, Elasticsearch is launched locally on port 9200. We first need to instantiate an Elasticsearch client object and connect to the service.

In [68]:
from elasticsearch import Elasticsearch

config = {'host':'localhost', 'port':9200}
es = Elasticsearch([config])

# test connection
es.ping()

True

Before we go further, let's introduce a few concepts that are specific to Elasticsearch and the process of indexing data. In Elasticsearch, an ***index*** is a collection of documents that have common characteristics (similar to a database schema in an RDBMS). ***Documents*** are JSON objects having their own set of key-value pairs consisting of various data types (similar to rows/fields in RDBMS). When we add a document into an index, the value for the document's text fields go through an analysis process prior to being indexed. This means that when executing a search query against an existing index, we are actually searching against the post-processed representation that is stored in the inverted index, not the raw input document itself.

![Elasticsearch Index Process](my_icons/elastic_index_process.png)
[Image Credit](https://medium.com/elasticsearch/introduction-to-analysis-and-analyzers-in-elasticsearch-4cf24d49ddab)

The anaysis process is a customizable pipeline carried out by a dedicated ***Analyzer***. Elasticsearch analyzers are composed of three sequential steps that form a processing pipeline: *character filters, a tokenizer, and token filters.* Each of these components modify the input stream of text according to some configurable settings. 
- **Character Filters:** First, character filters have the ability to add, remove, or replace specific items in the text field. A common application of this filter is to strip `html` markup from the raw input. 
- **Tokenizer:** After applying character filters, the transformed text is then passed to a tokenizer which breaks up the input string into individual tokens with a provided strategy. By default, the `standard` tokenizer splits tokens whenever it encounters a whitespace character, and also splits on most symbols (like commas, periods, semicolons, etc.)
- **Token Filters:** Finally, the token stream is passed to a token filter which acts to add, remove, or modify tokens. Typical token filters include `lowercase` which converts all tokens to lowercase form, and `stop` which removes commonly occuring tokens called *stopwords*. 

Elasticsearch comes with several built-in Analyzers that satisfy common use cases and defaults to the `Standard Analyzer`. The Standard Analyzer doesn't contain any character filters, uses a `standard` tokenizer, and applies a `lowercase` token filter. Let's take a look at [a borrowed example](https://medium.com/elasticsearch/introduction-to-analysis-and-analyzers-in-elasticsearch-4cf24d49ddab) sentence as its passed through this pipeline:

![Elasticsearch Analyzer Pipeline](my_icons/elasticsearch_standard_analyzer.png)
[Image Credit](https://medium.com/elasticsearch/introduction-to-analysis-and-analyzers-in-elasticsearch-4cf24d49ddab)

Crafting analyzers to your use case requires domain knowledge of the problem and dataset at hand, and doing so properly is key to optimizing relevance scoring for your search application. We found [this blog series](https://medium.com/elasticsearch/introduction-to-analysis-and-analyzers-in-elasticsearch-4cf24d49ddab) very useful in explaining the importance of analysis in Elasticsearch.

#### Create an Index


Let's create a new index and add our Wikipedia articles to it. To create an index, we provide a name and optionally some index configurations. Here we are specifying a set of `mappings` that indicate our anticipated index schema, data types, and how the text fields should be processed. If no `body` is passed, Elasticsearch will automatically infer fields and data types from incoming documents, as well as apply the `Standard Analyzer` to any text fields.

In [100]:
index_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                "standard_analyzer": {
                    "type": "standard"
                }
            }
        }
    },
    "mappings": {
        "dynamic": "strict", 
        "properties": {
            "document_title": {"type": "text", "analyzer": "standard_analyzer"},
            "document_text": {"type": "text", "analyzer": "standard_analyzer"}
            }
        }
    }

index_name = 'squad-standard-index'
es.indices.create(index=index_name, body=index_config, ignore=400)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'squad-standard-index'}

#### Populate the Index

We can then loop through our list of Wikipedia titles & articles and add them to our newly created Elasticsearch index.

In [101]:
from tqdm.notebook import tqdm

def populate_index(es_obj, index_name, evidence_corpus):
    '''
    Loads records into an existing Elasticsearch index

    Args:
        es_obj (elasticsearch.client.Elasticsearch)
        index_name (str)
        evidence_corpus (list) - list of dicts containing data records

    '''

    for i, rec in enumerate(tqdm(evidence_corpus)):
    
        try:
            index_status = es_obj.index(index=index_name, id=i, body=rec)
        except:
            print(f'Unable to load document {i}.')
            
    n_records = es_obj.count(index=index_name)['count']
    print(f'Succesfully loaded {n_records} into {index_name}')


    return

In [102]:
all_wiki_articles = wiki_articles + wiki_articles_dev

populate_index(es_obj=es, index_name='squad-standard-index', evidence_corpus=all_wiki_articles)


Succesfully loaded 20239 into squad-standard-index


#### Search the Index

Wahoo! We now have some documents loaded into into an index. Elasticsearch provides a rich [query language](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html) that supports a diverse range of query types. For this example, we'll use the standard query for performing full-text search called a `match` query. By default, Elasticsearch sorts and returns a JSON reponse of search results based on a computed a [relevance score](https://qbox.io/blog/practical-guide-elasticsearch-scoring-relevancy#:~:text=Together%2C%20these%20combine%20into%20a,number%20known%20as%20the%20_score.) which indicates how well a given document matches the query. Along with the relevance score of each matched document, the search response also includes the amount of time the query took to run.

Let's look at a simple `match` query used to search the `document_text` field in our newly created index.

> Note: As previously mentioned, all documents in the index have gone through an analysis process prior to indexing - this is called *index time analysis*. To maintain consistency in matching text queries against the post-processed index tokens, the same Analyzer used on a given field at index time is automatically applied to the query text at search time. *Search time analysis* is applied depending on which query type is used - `match` queries apply search time analysis by default.

In [103]:
def search_es(es_obj, index_name, query, n_results):
    '''
    Execute an Elasticsearch query on a specified index
    
    Args:
        es_obj - Elasticsearch client object
        index_name (str) - Name of index to query
        query (dict) - Query DSL
        n_results (int) - Number of results to return
        
    Returns
        res - Elasticsearch response object
    
    '''
    
    res = es_obj.search(index=index_name, body=query, size=n_results)
    
    return res

In [104]:
question_text = 'Who was the first president of the Republic of China?'

# construct query
query = {
        'query': {
            'match': {
                'document_text': question_text
                }
            }
        }

# execute query
res = search_es(es_obj=es, index_name='squad-standard-index', query=query, n_results=10)

In [105]:
print(f'Question: {question_text}')
print(f'Query Duration: {res["took"]} milliseconds')
print('Title, Relevance Score:')
[(hit['_source']['document_title'], hit['_score']) for hit in res['hits']['hits']]

Question: Who was the first president of the Republic of China?
Query Duration: 9 milliseconds
Title, Relevance Score:


[('Modern_history_54', 23.131157),
 ('Nanjing_18', 17.076923),
 ('Republic_of_the_Congo_10', 16.840765),
 ('Prime_minister_16', 16.137493),
 ('Korean_War_29', 15.801523),
 ('Korean_War_43', 15.586578),
 ('Qing_dynasty_52', 15.291815),
 ('Chinese_characters_55', 14.773873),
 ('Korean_War_23', 14.736045),
 ('2008_Sichuan_earthquake_48', 14.417962)]

# Evaluating Retriever Performance

Ok, so we now have a basic understanding of how to use Elasticsearch as an IR tool to return some results for a given question, but how do we know if it's working? How do we evaluate what a good IR tool looks like? Like we pointed out in the introduction of this post, if the Retriever component of our QA system doesn't provide the correct passage to the Reader, we are doomed from the start.

To evaluate how well our Retriever is working, we'll need two things: some labeled examples (i.e. SQuAD2.0 question/answer pairs) and some performance metrics. In the conventional world of information retrieval, there are [many metrics](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)) used to quantify the relevance of a set of query results - largely centered around the concepts of precision and recall. For IR in the context of question answering, we adapt some of these ideas into two metrics: *recall* and *mean average precision (mAP)*. Additionally, we evaluate the amount of time required to execute a query since the main point of having a two stage QA system is to efficiently narrow the large search space for our Reader.

#### Recall

Traditionally, recall in IR indicates the fraction of all relevant documents that are retrieved. In the context of end-to-end QA systems sitting on large corpora of documents, we are less concerned with finding *all* of the passages containing the answer (because it would take significant time to read through all of them anyway) and more concerned with the binary presence of a passage containing the correct answer being returned. In that light, we define a Retriever's recall across a set of questions as the *percentage of questions for which the answer segment appears in one of the top N pages returned by the search method.*

#### Mean Average Precision

While the *recall* metric focuses on the minimum viable result set to enable the Reader for success, we do still care about the composition of that result set. We want a metric that rewards a Retriever for a.) returning a lot of answer-containing documents in the result set (i.e. traditional meaning of precision) and b.) returning those answer-containing documents higher up in the result set than non-answer-containing documents (i.e. ranking them correctly). This is precisely (🙃) what mean average precision (mAP) does for us. 

To explain mAP further, let's first break down the concept of average precision for information retrieval. If our Retriever is asked to return N documents and the total number of those N documents that actually contains the answer is m, then average precision (AP) is defined as:

\begin{equation*}
AP@N = \frac{1}{m} \sum_{k=1}^N (P(k) \text{ if the }k^{th} \text{ item contains the answer)} = \frac{1}{m} \sum_{k=1}^N P(k)*rel(k)
\end{equation*}

where $rel(k)$ is just a binary indication of whether the $k^{th}$ passage contains the correct answer segment or not. Using a concrete example, consider retrieving $N=3$ documents, of which one actually contains the correct answer segment. Here are three scenarios for how this could happen:

| Scenario | Binary Indication | Precision @k's |     Average Precision @N     |
|:--------:|:-----------------:|:--------------:|:----------------------------:|
|     A    |     [1, 0, 0]     |   [1/1, 0, 0]  |   (1/1)*[(1/1) + 0 + 0] = 1  |
|     B    |     [0, 1, 0]     |   [0, 1/2, 0]  |  (1/1)*[0 + (1/2) + 0] = 0.5 |
|     C    |     [0, 0, 1]     |   [0, 0, 1/3]  | (1/1)*[0 + 0 + (1/3)] = 0.33 |

Despite the fact that in each scenario we only have one document containing the correct answer, Scenario A is rewarded with the highest score because it was able to correctly rank the ground truth document relative to the others returned. Since average precision is calculated on a per query basis, the mean average precision is simply just the average AP across all queries. 

Now using our Wikipedia passage index, let's define a function called `evaluate_retriever` to loop through all quesion/answer examples from the SQuAD2.0 train set and see how well our Elasticsearch retriever peforms in terms of recall, mAP, and averege query duration when retrieving $N=3$ passages.

In [133]:
import numpy as np
import pandas as pd

def average_precision(binary_results):
    
    ''' Calculates the average precision for a list of binary indicators '''
    
    m = 0
    precs = []

    for i, val in enumerate(binary_results):
        if val == 1:
            m += 1
            precs.append(sum(binary_results[:i+1])/(i+1))
            
    ap = (1/m)*np.sum(precs) if m else 0
            
    return ap


def evaluate_retriever(es_obj, index_name, qa_records, n_results):
    '''
    This function loops through a set of question/answer examples from SQuAD2.0 and 
    evaluates Elasticsearch as a information retrieval tool in terms of recall, mAP, and query duration.
    
    Args:
        es_obj
        index_name (str)
        qa_records (list) - list of qa_records from preprocessing steps
        n_results (int) - the number of results ElasticSearch should return for a given query
        
    Returns:
        test_results_df (pd.DataFrame) - a dataframe recording search results info for every example in qa_records
    
    '''
    
    results = []
    
    for i, qa in enumerate(tqdm(qa_records)):
        
        ex_id = qa['example_id']
        question = qa['question_text']
        answer = qa['short_answer']
        
        # construct and execute query
        query = {
                'query': {
                    'match': {
                        'document_text': question
                        }
                    }
                }
        
        res = search_es(es_obj=es_obj, index_name=index_name, query=query, n_results=n_results)
        
        # calculate performance metrics from query response info
        duration = res['took']
        binary_results = [int(answer.lower() in doc['_source']['document_text'].lower()) for doc in res['hits']['hits']]
        ans_in_res = int(any(binary_results))
        ap = average_precision(binary_results)

        rec = (ex_id, question, answer, duration, ans_in_res, ap)
        results.append(rec)
    
    # format results dataframe
    cols = ['example_id', 'question', 'answer', 'query_duration', 'answer_present', 'average_precision']
    results_df = pd.DataFrame(results, columns=cols)
    
    # format results dict
    metrics = {'Recall': results_df.answer_present.value_counts(normalize=True)[1],
               'Mean Average Precision': results_df.average_precision.mean(),
               'Average Query Duration':results_df.query_duration.mean()}
    
    return results_df, metrics



In [134]:
# combine train/dev examples and filter out SQuAD records that
# do not have a short answer for the given question
all_qa_records = qa_records+qa_records_dev
qa_records_answerable = [record for record in all_qa_records if record['short_answer'] != '']

# run evaluation
results_df, metrics = evaluate_retriever(es_obj=es, index_name='squad-standard-index', qa_records=qa_records_answerable, n_results=3)


In [135]:
metrics

{'Recall': 0.8226180336176131,
 'Mean Average Precision': 0.7524133234140888,
 'Average Query Duration': 3.0550841518506937}

# Improving Search Results with Custom Analyzers

Identifying a correct passage in the Top 3 results for 82% of the SQuAD questions in ~3.4 milliseconds per question is not too bad! But that means that we've effectively limited our overall QA system to an 82% upper bound on performance. How can we improve upon this?

One simple and obvious way to increase recall would be to just retrieve more passages. The following figure shows the effects of varying corpus size and result size on Elasticsearch retriever recall. As expected we see that the number of passages retrieved (i.e. *Top N*) has a dramatic impact on recall; a ~10-15 point jump from 1 to 3 passages returned, and ~5 point jump for each of the other tiers. We also see a gradual decrease in recall as corpus size increases, which isn't surprising.

![Elasticsearch Index Process](my_icons/Recall_v_corpussize.png)

While increasing the number of passages retrieved is effective, it also has implications on overall system performance as the already slow Reader now has to reason over more text. Instead, we can lean on best practices in the well explored domain of information retrieval.

Optimizing full-text search is a battle between precision (returning as few irrelevant documents as possible) and recall (returning as many relevant documents as possible). Matching only exact words in the question query results in high precision, however it misses out on many passages that could possibly be considered relevant. Rather we can cast a wider net by searching for terms that are not exactly the same as in the question query, but are related in some way. Let's explore a few tips and tricks for widening the net.

  
### Custom Analyzers

https://www.elastic.co/guide/en/elasticsearch/guide/2.x/languages.html

Earlier in the post we described Elasticsearch Analyzers as a way to deal with the nuances of human language. Analyzers provide a flexible and extensible method to tailor search for a given dataset. Here are a few custom Analyzer components Elasticsearch provides that can help cast a wider net.

- **Stopwords**: Stopwords are the most frequently occuring words in the English language (ex. "and", "the", "to", etc.) and add minimal semantic value to a piece of text. Common practice in information retrieval is to identify and remove them from documents to decrease the size of the index and increase relevance of search results. 

- **Stemming**: The English language is inflected - words can alter their written form to express different meanings. For example, 'sing', 'sings', 'sang', and 'singing' are written with slight differences, but really mean the same thing. Stemming is a technique that attempts to reduce words to their root form and consequently improve retrievability. Stemming algorithms exploit the fact that search intent is *usually* word-form agnostic - which is not always the case. For example, the phrases "fly fishing" and "flying fish" both stem to "fli fish" which would provide a very incorrect result depending on the users intent. Despite that, stemming does increase search recall by relaxing query term inflection. We'll implement the [Snowball](https://snowballstem.org/) stemming algorithm as a token filter in our custom Analyzer.

In [129]:
# create new index
index_config = {
    "settings": {
        "analysis": {
            "analyzer": {
                "stop_stem_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter":[
                        "lowercase",
                        "stop",
                        "snowball"
                    ]
                    
                }
            }
        }
    },
    "mappings": {
        "dynamic": "strict", 
        "properties": {
            "document_title": {"type": "text", "analyzer": "stop_stem_analyzer"},
            "document_text": {"type": "text", "analyzer": "stop_stem_analyzer"}
            }
        }
    }

es.indices.create(index='squad-stop-stem-index', body=index_config, ignore=400)

# populate the index
populate_index(es_obj=es, index_name='squad-stop-stem-index', evidence_corpus=all_wiki_articles)

# evaluate retriever performance
stop_stem_results_df, stop_stem_metrics = evaluate_retriever(es_obj=es, index_name='squad-stop-stem-index',\
                                                             qa_records=qa_records_answerable, n_results=3)



Succesfully loaded 20239 into squad-stop-stem-index


In [137]:
stop_stem_metrics

{'Recall': 0.8501115914996388,
 'Mean Average Precision': 0.7800892731997112,
 'Average Query Duration': 0.7684287701215108}

# Impact of Retriever in End-to-End QA System